<a href="https://colab.research.google.com/gist/zredlined/c42d71d7c94078e0ff9b864c1dd6ec24/privacy-safe-llm-training-for-financial-risk-analysis-azure-openai-gretel-synthetic-data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🔒 Safe Fine-Tuning of Azure OpenAI Models: Financial Risk Analysis with Gretel's Privacy-Preserving Synthetic Data

## Introduction

This notebook demonstrates how to safely train Large Language Models (LLMs) on sensitive financial data using privacy-preserving synthetic data. When working with regulated data in domains like finance, healthcare, or personal information, directly training models on raw data can pose significant privacy and compliance risks.

We solve this challenge using:
- 🤖 **[Gretel](https://gretel.ai)**: Synthetic data platform for generating training data for ML and AI use cases.  
- ☁️ **[Azure OpenAI Fine-tuning](https://learn.microsoft.com/en-us/azure/ai-services/openai/how-to/fine-tuning)**: Fine-tuning capabilities for customizing LLMs to specific domains
- 📊 **Financial Risk Analysis**: A practical use case showing how to train models to analyze financial documents for risk assessment

## 🛠️ What We'll Build

We'll create a financial risk analysis model that can:
- Process financial documents and regulatory filings
- Identify potential risks and exposures
- Generate structured risk assessments in JSON format
- Maintain data privacy compliance through synthetic training data

## 📈 The Data

We're using Gretel's [`gretel-financial-risk-analysis-v1`](https://huggingface.co/datasets/gretelai/gretel-financial-risk-analysis-v1) dataset, which contains:
- Synthetic financial documents and regulatory filings
- Corresponding risk assessments and analysis
- Built-in privacy guarantees through differential privacy
- Real-world patterns while protecting sensitive information

## 📚 Notebook Structure
1. Set up Azure OpenAI credentials and environment
2. Load and prepare the Gretel synthetic dataset
3. Fine-tune an Azure OpenAI model
4. Monitor training progress
5. Evaluate the resulting model

This approach enables organizations to leverage LLMs for financial analysis while maintaining strict data privacy and regulatory compliance. 🚀

In [ ]:
# Install dependencies

!pip install -U -qq openai gretel_client datasets

In [ ]:
# Load secrets and create our Azure client

import os
from getpass import getpass
from openai import AzureOpenAI

os.environ['AZURE_OPENAI_ENDPOINT'] = "https://YOUR_ENDPOINT.openai.azure.com/"
os.environ['AZURE_OPENAI_API_KEY'] = getpass("Enter your API key: ")

azure_client = AzureOpenAI(api_version="2024-08-01-preview")

In [ ]:
from datasets import load_dataset

from gretel_client.fine_tuning import OpenAIFormatter, OpenAIFineTuner

HF_DATASET_NAME = "gretelai/gretel-financial-risk-analysis-v1"

# All metadata related to the fine-tuning job will be stored here
CHECKPOINT_FILE = "openai_checkpoint.json"

In [ ]:
# Load the synthetic dataset from Huggingface. Any Training and Validation DataFrame can be used here.

dataset = load_dataset(HF_DATASET_NAME)
train_df = dataset["train"].to_pandas()
validation_df = dataset["test"].to_pandas()
train_df.head()

In [ ]:
# Create a formatter object to convert the DataFrame into a format that OpenAI can understand.

# When creating the formatter object, you can specify the columns that you want to use for the input and output as string template variables.
# The string template variables will be replaced with the actual values from the DataFrame when the training dataset is created and if 
# any of the string template variables are not found in the DataFrame, an error will be raised.

# For this specific example, we do not need to do any special formatting from the DataFrame, so we only specify string template variables
# directly with no other text.

SYSTEM_MESSAGE = """You are an expert financial risk analyst. Analyze the provided text for financial risks,
    and output a structured assessment in JSON format including risk detection, specific risk flags,
    financial exposure details, and analysis notes."""

# This formatter will be provided to our fine tuning object later on.
formatter = OpenAIFormatter(system_message=SYSTEM_MESSAGE, user_template="{input}", assistant_template="{output}")

# We can take a peek at what the formatter will do to the DataFrame
formatter.peek_ft_dataset(input_dataset=train_df, n=2)


In [ ]:
# Next, we create our Fine Tuning Adapter. We will use this instance throughout the rest of the tutorial.

fine_tuner = OpenAIFineTuner(
    openai_client=azure_client,
    formatter=formatter,
    train_data=train_df,
    validation_data=validation_df
)

In [ ]:
# As we progress through the tutorial, we will store our checkpoint occasioonally to disk. 
# The fine tuner instnace can be reloaded from this checkpoint by providing it as a kwarg to the constructor:

# fine_tuner = OpenAIFineTuner(openai_client=azure_client, checkpoint=CHECKPOINT_FILE)

In [ ]:
# First, we will prepare our training and validation datasets and upload them to the OpenAI Service. This is handled automatically by the fine tuner.

fine_tuner.prepare_and_upload_data()
fine_tuner.save_checkpoint(CHECKPOINT_FILE) # save our file IDs to disk

In [ ]:
# Next, we will start fine tuning our model.
# By default, this method will wait and accumulate the training event logs.
# If you terminate this cell, see the next cell for how to re-attach to the job.

fine_tuner.start_fine_tuning(model="gpt-4o-mini-2024-07-18", epochs=1, checkpoint_save_path=CHECKPOINT_FILE)
fine_tuner.save_checkpoint(CHECKPOINT_FILE) # save our file IDs to disk

In [ ]:
# Re-attach to the fine tuning job. This will load and display all available logging events from the job.

fine_tuner.wait_for_fine_tune_job(checkpoint_save_path=CHECKPOINT_FILE)

In [ ]:
fine_tuner.graph_training_metrics()

In [ ]:
# Next, with our fine-tuned model, we can create a deployment so we can run inference.
# To run a deployment, you can use the Azure shell. Run this cell to get the full CLI command to use.

# NOTE: This will be the "model" name that is used in subsequent chat completions
deployment_name = "risk-analysis-gpt"

# Retrieve this information from the Azure AI Studio portal.
resource_group = "gretel-fine-tuning-dev"
resource_name = "gretel-fine-tuning-dev"

print(f"""Model can be deployed via Azure shell command: \n\n az cognitiveservices account deployment create \\
    --resource-group {resource_group} \\
    --name {resource_name} \\
    --deployment-name {deployment_name} \\
    --model-name {fine_tuner.checkpoint.open_ai_fine_tuned_model_id} \\
    --model-version "1" \\
    --model-format OpenAI \\
    --sku-capacity "1" \\
    --sku-name "Standard"
""".format(resource_group=resource_group, resource_name=resource_name, deployment_name=deployment_name))

In [ ]:
# With our model deployed, let's create a dataset we'd like to send to our fine-tuned model.

test_cases = [
    # Case 1: High financial risk scenario
    """
    The Company has entered into a five-year contract to purchase raw materials
    from a single supplier in a volatile market. The contract requires minimum
    purchases of $10M annually with no cancellation clause. Recent market analysis
    suggests potential price fluctuations of up to 40% in the next year.
    """,

    # Case 2: Moderate financial risk scenario
    """
    Company XYZ announced a major expansion into emerging markets, requiring
    $50M in upfront capital expenditure. The project will be funded through
    a combination of variable-rate loans (60%) and existing cash reserves.
    Market analysts expect interest rates to rise by 2% over the next year.
    """,

    # Case 3: No financial risk scenario
    """
    The company has successfully completed its annual employee satisfaction survey
    with a 95% participation rate. Results show high employee engagement scores
    across all departments. The HR department is planning to implement new
    professional development programs next quarter, which will be covered by
    the existing training budget.
    """
]

# With our existing formatter, we can also create a dataset that can be used for completion tasks.
# The `user_messages` parameter is a list of dictionaries where each dictionary contains the input message.
# The dictionary keys should match the string template variables in the formatter that were earlier specified and 
# are also columns in the training dataset.
messages = formatter.create_completion_dataset(user_messages=[{"input": prompt} for prompt in test_cases])
messages

In [ ]:
responses = fine_tuner.create_chat_completitions(deployment_name, messages=messages, model_params={"temperature": 0}, parse_json=True)
responses